In [2]:
from utils.model_generator import get_model
import torch
import json
import nibabel as nib
import numpy as np


weights_path = 'models/Test_4_New_Metrics/custom_resnet152/Adam/150/best_model_custom_resnet152.pth'

file_path = ["output/derivatives/preprocessed/sub-RC4101/ses-1/anat/sub-RC4101_ses-1_T1w_brain_normalized.nii.gz"]

image = nib.load(file_path[0]).get_fdata()
image = image.astype(np.float32)
image = np.expand_dims(image, axis=0)
image = np.expand_dims(image, axis=0)
image = torch.from_numpy(image)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = torch.load(weights_path)

model = get_model('custom_resnet152').to(device)

model.load_state_dict(weights)

inference_dict = {}

with torch.inference_mode():
    model.eval()

    example = image.to(device)
    output = model(example)

logits = torch.softmax(output, dim=1)
inference = torch.argmax(logits, dim=1)
print(logits, inference)

inference_dict[file_path[0].split("/")[-1]] = logits.cpu().detach().numpy().tolist(), inference.cpu().detach().numpy().tolist()

with open('inference/labels.json', 'w',encoding="utf") as f:
    json.dump(inference_dict, f, indent=4)


/tmp/ipykernel_1730465/3973701282.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path)


[INFO] create new resnet152 model.
tensor([[0.9963, 0.0037]], device='cuda:0') tensor([0], device='cuda:0')


In [2]:
import torchinfo

torchinfo.summary(model,(1, 1, 91, 109, 91), col_names=["num_params","trainable"]
                  ,row_settings=["var_names"])

Layer (type (var_name))                       Param #                   Trainable
CustomEfficientNet (CustomEfficientNet)       --                        Partial
├─PreprocessInput (preprocess)                --                        True
│    └─Sequential (conv_block_1)              --                        True
│    │    └─Conv3d (0)                        900                       True
│    │    └─ReLU (1)                          --                        --
│    │    └─Conv3d (2)                        40,050                    True
│    │    └─ReLU (3)                          --                        --
│    │    └─MaxPool3d (4)                     --                        --
│    └─Sequential (conv_block_2)              --                        True
│    │    └─Conv3d (0)                        10,025                    True
│    │    └─ReLU (1)                          --                        --
│    │    └─Conv3d (2)                        2,613                     True

In [5]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/Test_4_New_Metrics/custom_resnet152/Adam/150/best_model_custom_resnet152')
writer.add_graph(model, torch.randn(1, 1, 91, 109, 91).to(device))
writer.close()